# Joins

## Prerrequisites

Install Spark and Java in VM

In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark 3.5.0
!wget -q https://apache.osuosl.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [ ]:
ls -l # check the .tgz is there

total 267684
drwxr-xr-x 1 root root      4096 Dec  6 14:35 sample_data/
-rw-r--r-- 1 root root 274099817 Oct 15 10:53 spark-3.3.1-bin-hadoop2.tgz


In [ ]:
# unzip it
!tar xf spark-3.5.0-bin-hadoop3.tgz

In [ ]:
!pip install -q findspark

In [ ]:
!pip install py4j

# For maps
!pip install folium
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Define the environment

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Start Spark Session 

---

In [ ]:
import findspark
findspark.init("spark-3.5.0-bin-hadoop3")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .appName("Joins") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.3.1'

In [ ]:
spark

In [ ]:
# Import sql functions
from pyspark.sql.functions import *

Download datasets

In [ ]:
!mkdir -p dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/guitars.json -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/guitarPlayers.json -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/bands.json -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/employees.csv -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/salaries.csv -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/deptmanagers.csv -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/titles.csv -P /dataset
!ls /dataset

Read JSONs

In [ ]:
guitarsDF = spark.read \
    .option("inferSchema", "true") \
    .json("/dataset/guitars.json")

guitaristsDF = spark.read \
    .option("inferSchema", "true") \
    .json("/dataset/guitarPlayers.json")

bandsDF = spark.read \
    .option("inferSchema", "true") \
    .json("/dataset/bands.json")

In [ ]:
guitarsDF.show(3)
guitaristsDF.show(3)
bandsDF.show(3)


+--------------------+---+------+------------+
|          guitarType| id|  make|       model|
+--------------------+---+------+------------+
|Electric double-n...|  0|Gibson|    EDS-1275|
|            Electric|  5|Fender|Stratocaster|
|            Electric|  1|Gibson|          SG|
+--------------------+---+------+------------+
only showing top 3 rows

+----+-------+---+------------+
|band|guitars| id|        name|
+----+-------+---+------------+
|   0|    [0]|  0|  Jimmy Page|
|   1|    [1]|  1| Angus Young|
|   2| [1, 5]|  2|Eric Clapton|
+----+-------+---+------------+
only showing top 3 rows

+-----------+---+------------+----+
|   hometown| id|        name|year|
+-----------+---+------------+----+
|     Sydney|  1|       AC/DC|1973|
|     London|  0|Led Zeppelin|1968|
|Los Angeles|  3|   Metallica|1981|
+-----------+---+------------+----+
only showing top 3 rows



## Examples

Inner Join =  everything from BOTH DFs for which there is a row in the right DF satisfying the condition


In [ ]:
joinCondition = guitaristsDF.band == bandsDF.id
guitaristsBandsDF = guitaristsDF.join(bandsDF, joinCondition, "inner")
guitaristsBandsDF.show(3)

+----+-------+---+------------+-----------+---+------------+----+
|band|guitars| id|        name|   hometown| id|        name|year|
+----+-------+---+------------+-----------+---+------------+----+
|   1|    [1]|  1| Angus Young|     Sydney|  1|       AC/DC|1973|
|   0|    [0]|  0|  Jimmy Page|     London|  0|Led Zeppelin|1968|
|   3|    [3]|  3|Kirk Hammett|Los Angeles|  3|   Metallica|1981|
+----+-------+---+------------+-----------+---+------------+----+



In [ ]:
# things to bear in mind
# guitaristsBandsDF.select("id", "band").show(3) # will crash because there are two columns named "id"

In [ ]:
# you can rename the column
bandsDFMod = bandsDF.withColumnRenamed("id", "bandId") \
    .withColumnRenamed("name", "bandName")
guitaristsBandsDF2 = guitaristsDF.join(bandsDFMod,
    guitaristsDF.band == bandsDFMod.bandId, "inner").orderBy("name")
guitaristsBandsDF2.show(3)

# there are other options but it is the best practice if it is possible

+----+-------+---+------------+-----------+------+------------+----+
|band|guitars| id|        name|   hometown|bandId|    bandName|year|
+----+-------+---+------------+-----------+------+------------+----+
|   1|    [1]|  1| Angus Young|     Sydney|     1|       AC/DC|1973|
|   0|    [0]|  0|  Jimmy Page|     London|     0|Led Zeppelin|1968|
|   3|    [3]|  3|Kirk Hammett|Los Angeles|     3|   Metallica|1981|
+----+-------+---+------------+-----------+------+------------+----+



Left Outer = everything in the inner join + all the rows in the LEFT DF, with nulls in where the data is missing

In [ ]:
joinCondition2 = guitaristsDF.band == bandsDFMod.bandId
guitaristsDF.join(bandsDFMod, joinCondition2, "left_outer").orderBy("name").show(3)
# see nulls on Eric Calpton

+----+-------+---+------------+--------+------+------------+----+
|band|guitars| id|        name|hometown|bandId|    bandName|year|
+----+-------+---+------------+--------+------+------------+----+
|   1|    [1]|  1| Angus Young|  Sydney|     1|       AC/DC|1973|
|   2| [1, 5]|  2|Eric Clapton|    null|  null|        null|null|
|   0|    [0]|  0|  Jimmy Page|  London|     0|Led Zeppelin|1968|
+----+-------+---+------------+--------+------+------------+----+
only showing top 3 rows



  Right Outer = everything in the inner join + all the rows in the RIGHT DF, with nulls in where the data is missing


In [ ]:
guitaristsDF.join(bandsDFMod, joinCondition2, "right_outer").orderBy("name").show(3)
# see nulls for The Beatles

+----+-------+----+-----------+---------+------+------------+----+
|band|guitars|  id|       name| hometown|bandId|    bandName|year|
+----+-------+----+-----------+---------+------+------------+----+
|null|   null|null|       null|Liverpool|     4| The Beatles|1960|
|   1|    [1]|   1|Angus Young|   Sydney|     1|       AC/DC|1973|
|   0|    [0]|   0| Jimmy Page|   London|     0|Led Zeppelin|1968|
+----+-------+----+-----------+---------+------+------------+----+
only showing top 3 rows



 Outer Join = everything in the inner join + all the rows in BOTH DFs, with nulls in where the data is missing


In [ ]:
guitaristsDF.join(bandsDFMod, joinCondition2, "outer").orderBy("name").show(3)
# see nulls for The Beatles and Eric Clapton

+----+-------+----+------------+---------+------+-----------+----+
|band|guitars|  id|        name| hometown|bandId|   bandName|year|
+----+-------+----+------------+---------+------+-----------+----+
|null|   null|null|        null|Liverpool|     4|The Beatles|1960|
|   1|    [1]|   1| Angus Young|   Sydney|     1|      AC/DC|1973|
|   2| [1, 5]|   2|Eric Clapton|     null|  null|       null|null|
+----+-------+----+------------+---------+------+-----------+----+
only showing top 3 rows



  Semi-joins = everything in the LEFT DF for which there is a row in the right DF satisfying the condition


In [ ]:
guitaristsDF.join(bandsDFMod, joinCondition2, "left_semi").orderBy("name").show(3)


+----+-------+---+------------+
|band|guitars| id|        name|
+----+-------+---+------------+
|   1|    [1]|  1| Angus Young|
|   0|    [0]|  0|  Jimmy Page|
|   3|    [3]|  3|Kirk Hammett|
+----+-------+---+------------+



Anti-joins = everything in the LEFT DF for which there is NO row in the right DF satisfying the condition


In [ ]:
guitaristsDF.join(bandsDFMod, joinCondition2, "anti").orderBy("name").show(3)

+----+-------+---+------------+
|band|guitars| id|        name|
+----+-------+---+------------+
|   2| [1, 5]|  2|Eric Clapton|
+----+-------+---+------------+



Cross-join = takes every instance in the LEFT DF and create a new one for every intance in the right DF (not very common)

In [ ]:
guitaristsDF.crossJoin(bandsDFMod).show(20)

+----+-------+---+------------+-----------+------+------------+----+
|band|guitars| id|        name|   hometown|bandId|    bandName|year|
+----+-------+---+------------+-----------+------+------------+----+
|   0|    [0]|  0|  Jimmy Page|     Sydney|     1|       AC/DC|1973|
|   1|    [1]|  1| Angus Young|     Sydney|     1|       AC/DC|1973|
|   2| [1, 5]|  2|Eric Clapton|     Sydney|     1|       AC/DC|1973|
|   3|    [3]|  3|Kirk Hammett|     Sydney|     1|       AC/DC|1973|
|   0|    [0]|  0|  Jimmy Page|     London|     0|Led Zeppelin|1968|
|   1|    [1]|  1| Angus Young|     London|     0|Led Zeppelin|1968|
|   2| [1, 5]|  2|Eric Clapton|     London|     0|Led Zeppelin|1968|
|   3|    [3]|  3|Kirk Hammett|     London|     0|Led Zeppelin|1968|
|   0|    [0]|  0|  Jimmy Page|Los Angeles|     3|   Metallica|1981|
|   1|    [1]|  1| Angus Young|Los Angeles|     3|   Metallica|1981|
|   2| [1, 5]|  2|Eric Clapton|Los Angeles|     3|   Metallica|1981|
|   3|    [3]|  3|Kirk Hammett|Los

*Union Join is just an Union (note it only works if both DFs have the same schema)

## Exercises
1. Read employees.csv, deptmanagers.csv, salaries.csv and titles.csv to a DFs.
2. Show all employees and their max salary (there can be different salaries registered for the same employee)
3. Show all employees who were never managers. Check it by getting all the managers and checking that they are not in the table
4. Find the job titles of the best paid 10 employees in the company (note that there can be different titles registered for the same employee)

Exercise 1

Exercise 2

Exercise 3

Exercise 4